In [40]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')


<h1 align="center">Synchrotron Tools</h1>

<h2 align="left">TOOL 1: PV calculator</h2>

In [41]:
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension

import ipywidgets as widgets
import numpy as np
import sys

from ipywidgets import Layout, HBox, VBox
from IPython.core.display import display
from scipy.optimize import fsolve

global P_max, phase, EoS
P_max, V0, EoS = 200, 40, 1

def phase_selector(phase):
    global V0, K, Kp, a0, b0, c0
    if phase == 2:
        dropdown2.value, dropdown2.disabled = 2, True 
        #Data from: Tange et al. (2012) 10.1029/2011JB008988 and Tange et al. (2009) 10.1029/2008JB005891
        V0, K, Kp, a0, b0, c0 = 162.373, 258.4, 4.1, 4.7769, 4.9284, 6.8972
        initialise()
    if phase == 3: 
        #Data from: Fiquet et al. (2002) American Mineralogist 87:1261-1265
        dropdown2.value, dropdown2.disabled = 2, True 
        V0, K, Kp, a0, b0, c0 = 279.2, 108, 5, 4.6377, 4.6377, 15.007
        initialise()
    if phase == 4:
        dropdown2.value, dropdown2.disabled = 1, False
    if phase == 5:
        #Data from: Dorogokupets & Dewaele (2007) 10.1080/08957950701659700
        dropdown2.value, dropdown2.disabled = 3, True 
        V0, K, Kp, a0, b0, c0 = 60.38, 277.3, 5.12, 3.9231, 3.9231, 3.9231
        initialise()
    if phase == 6:
        dropdown2.value, dropdown2.disabled = 1, False
    if phase == 7:
        #Data from: Dorogokupets & Dewaele (2007) 10.1080/08957950701659700
        dropdown2.value, dropdown2.disabled = 3, True 
        V0, K, Kp, a0, b0, c0 = 179.44, 23.83, 5.09, 5.6404, 5.6404, 5.6404
        initialise()
    if phase == 8:
        #Data from: Dorogokupets & Dewaele (2007) 10.1080/08957950701659700
        dropdown2.value, dropdown2.disabled = 3, True 
        V0, K, Kp, a0, b0, c0 = 40.73, 29.72, 5.14, 3.4407, 3.4407, 3.4407
        initialise()
                
def EoS_selector(EoS,phase):
    global V0, K, Kp, a0, b0, c0
    if phase == 4:
        #Data from: Wang et al. (2012) 10.1029/2011JB009100
        if EoS == 2:
            V0, K, Kp, a0, b0, c0 = 46.55, 294, 4.85, 4.66, 4.66, 2.8403
            initialise()
        if EoS == 3:
            V0, K, Kp, a0, b0, c0 = 46.55, 292, 5.01, 4.66, 4.66, 2.8403
            initialise()
        if EoS == 1:
            K_box.value, Kp_box.value, V0_box.value, VV_box.value, Va_box.value, Vb_box.value, Vc_box.value  = (''), (''), (''), (''), (''), (''), ('')
    if phase == 6:
        #Data from: Fei et al. (2012) 10.1073/pnas.0609013104
        if EoS == 2:
            V0, K, Kp, a0, b0, c0 = 67.85, 167, 5.77, 4.0786, 4.0786, 4.0786
            initialise()
        if EoS == 3:
            V0, K, Kp, a0, b0, c0 = 67.85, 167, 6.00, 4.0786, 4.0786, 4.0786
            initialise()
        if EoS == 1:
            K_box.value, Kp_box.value, V0_box.value, VV_box.value, Va_box.value, Vb_box.value, Vc_box.value  = (''), (''), (''), (''), (''), (''), ('')
            
def initialise():
    K_box.value = ("K = " + str(K) + " GPa")
    Kp_box.value = ("K' = " + str(Kp))
    V0_box.value = ("V₀ = " + str(V0) + " Å³")
    VV_box.value = ("V = " + str(V0) + " Å³")
    Va_box.value = ("a = " + str(a0) + " Å")
    Vb_box.value = ("b = " + str(b0) + " Å")
    Vc_box.value = ("c = " + str(c0) + " Å")
    P_slider.value = 0
        

def BM3(V,P):
    return P-((3/2)*K*(((V0/V)**(7/3))-((V0/V)**(5/3)))*(1-(3/4)*(4-Kp))*((V0/V)**(2/3)-1))

def Vinet(V,P):
    return P-(3*K*((1-((V/V0)**(1/3)))/((V/V0)**(2/3)))*np.exp((3/2)*(Kp-1)*(1-((V/V0)**(1/3)))))
            
dropdown1 = widgets.Dropdown(options = {'Choose': 1, 'MgSiO3 perovskite': 2, 'MgCO3 magnesite': 3, 'SiO2 stishovite': 4, 'Pt': 5, 'Au': 6, 'NaCl B1': 7, 'NaCl B2': 8},description='Phase:',layout=Layout(width='47.5%'))
dropdown2 = widgets.Dropdown(options = {'Choose': 1, '3rd Order Birch-Murnaghan': 2, 'Vinet': 3},description='EoS:',layout=Layout(width='47.5%'))    

w1=widgets.interactive(phase_selector,phase=dropdown1)
w2=widgets.interactive(EoS_selector,EoS=dropdown2,phase=dropdown1)

K_box = widgets.Text(disabled=True,text_color='green',background_color='green',layout=Layout(width='12%'))
Kp_box = widgets.Text(disabled=True,text_color='green',background_color='green',layout=Layout(width='12%'))
V0_box = widgets.Text(disabled=True,text_color='green',background_color='green',layout=Layout(width='12%'))

VBox([HBox([dropdown1]), HBox([dropdown2, K_box, Kp_box, V0_box])])

In [42]:
global P_max, phase, EoS, V0, K, Kp, a0, b0, c0
def V_calc(P):
        if EoS == 1 or 2:
            V = fsolve(lambda V,P: BM3(V,P), 1, args=(P), xtol=1e-5)
        if EoS == 3:
            V = fsolve(lambda V,P: Vinet(V,P), 1, args=(P), xtol=1e-5)
        VV_box.value = ("V = " + str(np.around(V,3))[1:-1] + " Å³")
        Va_box.value = ("a = " + str(np.around((V/V0)**(1/3)*a0,3))[1:-1] + " Å")
        Vb_box.value = ("b = " + str(np.around((V/V0)**(1/3)*b0,3))[1:-1] + " Å")
        Vc_box.value = ("c = " + str(np.around((V/V0)**(1/3)*c0,3))[1:-1] + " Å")
              
P_slider = widgets.FloatSlider(min=0, max=200, step=0.1, value=0, description='P in GPa:',slider_color='green',layout=Layout(width='100%'))            
widgets.interactive(V_calc,P=P_slider)

VV_box = widgets.Text(disabled=True,color='black',layout=Layout(width='19%'),description='Lattice:')
Va_box = widgets.Text(disabled=True,color='black',layout=Layout(width='12%'))
Vb_box = widgets.Text(disabled=True,color='black',layout=Layout(width='12%'))
Vc_box = widgets.Text(disabled=True,color='black',layout=Layout(width='12%'))
#box_titles = ['VV_box', 'Va_box','Vb_box','Vc_box']
#boxes = [widgets.Text(disabled=True,color='black',layout=Layout(width='12%')) for i in box_titles]

VBox([HBox([P_slider]), HBox([VV_box, Va_box, Vb_box, Vc_box])])

<h2 align="left">TOOL 2: Ruby Fluorescence and Raman Pressure Calibrations</h2>

In [43]:
def ruby_scale(scale1):
    global A, B
    if scale1 == 2:
        #Data from: Dewaele et al. (2004) 10.1103/PhysRevB.70.094112
        A, B = 1904, 9.500
        initialise2()
    if scale1 == 3:
        #Data from: Mao et al. (1986)
        A, B = 1904, 7.665
        initialise2()

def R1_0(R1_0_value):
    return R1_0

def ruby(R1):
    R1_0,V1_0,R1,V1=get_boxes()
    delta=R1-R1_0
    ruby_P=A/B*((1+delta/R1_0)**B-1)
    P_slider2.value=ruby_P
    raman_slider.value=(ruby_P*(1/0.501))+V1_0

def raman_scale(scale2):
    return scale2

def V1_0(V1_0_value):
    return V1_0_value

def raman(V1):
    R1_0,V1_0,R1,V1=get_boxes()
    raman_P=0.501*(V1-V1_0)
    P_slider2.value=raman_P
    ruby_slider.value=R1_0*(((B*((A/B)+raman_P))/A)**(1/B))
    
def get_boxes():
    R1_0   = (float(R1_0_box.value[0:-3]))
    V1_0   = (float(V1_0_box.value[0:-5]))
    R1     = ruby_slider.value
    V1     = raman_slider.value
    return (R1_0,V1_0,R1,V1)

def initialise2():
    R1_0,V1_0,R1,V1=get_boxes()
    ruby_slider.max    = 750
    ruby_slider.value  = R1_0
    P_slider2.max      = A/B*((1+(ruby_slider.max-R1_0)/R1_0)**B-1)
    P_slider2.value    = 0
    raman_slider.max   = ((P_slider2.max*(1/0.501))+V1_0)
    raman_slider.value = V1_0

ruby_dropdown = widgets.Dropdown(options = {'Choose': 1, 'Hydrostatic - Dewaele et al. 2004': 2, 'Non-hydrostatic - Mao et al. 1986': 3},description='RUBY:',layout=Layout(width='35%'))
raman_dropdown = widgets.Dropdown(options = {'Walter et al. 2014': 1},description='RAMAN:',layout=Layout(width='35%'),disabled=True)

R1_0_box = widgets.Text(value='694.2 nm',disabled=False,text_color='green',background_color='green',layout=Layout(width='18%'),description='R1 λ₀:')
V1_0_box = widgets.Text(value='1332 cm⁻¹',disabled=False,text_color='green',background_color='green',layout=Layout(width='18%'),description='D ν₀:')

ruby_slider = widgets.FloatSlider(min=694.2, max=750, step=0.01, value=694.2, description='R1 λ:',slider_color='green',layout=Layout(width='52.5%'))            
raman_slider = widgets.FloatSlider(min=1332, max=1600, step=0.1, value=1332, description='001 ν:',slider_color='green',layout=Layout(width='52.5%'))            

P_slider2 = widgets.FloatSlider(min=0, max=200, step=0.1, value=0, description='P in GPa:',slider_color='green',layout=Layout(width='100%'),disabled=True)            

w3=widgets.interactive(ruby_scale,scale1=ruby_dropdown)
w4=widgets.interactive(raman_scale,scale2=raman_dropdown)
w5=widgets.interactive(R1_0,R1_0_value=R1_0_box)
w6=widgets.interactive(V1_0,V1_0_value=V1_0_box)
w7=widgets.interactive(ruby,R1=ruby_slider)
w8=widgets.interactive(raman,V1=raman_slider)

ruby_dropdown.value = 1

VBox([HBox([ruby_dropdown,R1_0_box,ruby_slider]),HBox([raman_dropdown,V1_0_box,raman_slider]),HBox([P_slider2])])